# Identificazione delle "Golden Labels" per la Creazione del Test Set

**Obiettivo dello script:**
Questo script è fondamentale per suddividere il nostro dataset in un set di addestramento/validazione e un set di test affidabile. L'idea è quella di creare un **test set "golden"**, ovvero un insieme di dati la cui validità è corroborata da una fonte esterna. Per fare ciò, confrontiamo il nostro dataset (con sentiment generato tramite VADER) con il dataset **EmoSign**, che è stato validato manualmente. Le corrispondenze trovate tra i due dataset costituiranno le nostre "golden labels" e verranno usate esclusivamente per il testing.

**Funzionamento dello script:**

1.  **Caricamento dei Dataset:**

    - **Dataset EmoSign (da Hugging Face):** Viene caricato il dataset `catfang/emosign`. Questo è considerato la nostra "ground truth" o fonte di validazione, poiché contiene video di utterance già selezionati e usati in studi precedenti.
    - **Dataset Locale (con Sentiment):** Viene caricato il nostro file CSV locale (`asllrp_video_sentiment_data_with_neutral_0.34.csv`), che contiene le etichette di sentiment (Positivo, Negativo, Neutro) che abbiamo generato nello script precedente.

2.  **Estrazione degli ID Comuni:**

    - Da entrambi i dataset, viene estratto un identificatore univoco per ogni video, l'`utterance_id`. Questo ID permette di collegare in modo univoco i video tra i due dataset.
    - Per il dataset EmoSign, l'ID viene estratto dal nome del file.
    - Per il nostro dataset locale, l'ID corrisponde al nome del file senza l'estensione `.mp4`.

3.  **Identificazione delle "Golden Labels":**

    - Lo script confronta gli insiemi di `utterance_id` dei due dataset e trova l'intersezione. Le righe nel nostro dataset locale i cui `utterance_id` sono presenti anche nel dataset EmoSign vengono identificate come **"golden labels"**.
    - Queste sono le etichette di cui ci fidiamo di più, poiché confermate da una fonte esterna.

4.  **Creazione e Salvataggio dei Nuovi Dataset:**
    - **Test Set (Golden Labels):** Le "golden labels" vengono salvate in un nuovo file CSV (`golden_label_sentiment_with_neutral.csv`). Questo file sarà utilizzato **esclusivamente come test set** per valutare le performance finali dei modelli.
    - **Training/Validation Set:** Tutte le altre righe del nostro dataset locale (quelle che _non_ sono golden labels) vengono salvate in un altro file CSV (`..._without_golden.csv`). Questo file verrà utilizzato per l'**addestramento e la validazione** dei modelli, garantendo una separazione netta tra i dati di training e quelli di test.


In [16]:
# Step 2: Caricare i dataset e preparare gli ID
import pandas as pd
import os

# --- Caricamento e preparazione del dataset EmoSign da Hugging Face ---
print("Caricamento del dataset EmoSign da Hugging Face...")
try:
    # Per accedere a questo dataset, potrebbe essere necessario un login tramite `huggingface-cli login`
    df_emosign = pd.read_csv("hf://datasets/catfang/emosign/emosign_dataset.csv")
    print("Dataset EmoSign caricato con successo.")

    # Estrai l'ID utterance dalla colonna 'video_name'
    # L'ID è la parte numerica dopo il trattino basso '_'
    df_emosign["utterance_id"] = (
        df_emosign["video_name"].str.split("_").str[-1].astype(str)
    )
    emosign_ids = set(df_emosign["utterance_id"])
    print(f"Trovati {len(emosign_ids)} ID unici nel dataset EmoSign.")

except Exception as e:
    print(f"Errore durante il caricamento del dataset da Hugging Face: {e}")
    print(
        "Assicurati di aver effettuato l'accesso con 'huggingface-cli login' nel tuo terminale."
    )
    emosign_ids = set()

# --- Caricamento e preparazione del dataset locale ---
if emosign_ids:
    print("\nCaricamento del dataset locale con sentiment...")
    # local_csv_path = "../data/processed/asllrp_video_sentiment_data_0.34.csv" # Vecchio path
    local_csv_path = "../data/processed/asllrp_video_sentiment_data_with_neutral_0.4.csv"  # Nuovo path con neutrali
    try:
        df_local = pd.read_csv(local_csv_path)
        print("Dataset locale caricato con successo.")

        # Estrai l'ID utterance dalla colonna 'video_name'
        # Rimuovi l'estensione '.mp4'
        df_local["utterance_id"] = (
            df_local["video_name"].str.replace(".mp4", "", regex=False).astype(str)
        )
        local_ids = set(df_local["utterance_id"])
        print(f"Trovati {len(local_ids)} ID unici nel dataset locale.")

    except FileNotFoundError:
        print(f"Errore: File non trovato a '{local_csv_path}'")
        df_local = pd.DataFrame()
else:
    print("\nSalto il caricamento del dataset locale a causa di errori precedenti.")
    df_local = pd.DataFrame()

Caricamento del dataset EmoSign da Hugging Face...
Dataset EmoSign caricato con successo.
Trovati 200 ID unici nel dataset EmoSign.

Caricamento del dataset locale con sentiment...
Dataset locale caricato con successo.
Trovati 2118 ID unici nel dataset locale.
Dataset EmoSign caricato con successo.
Trovati 200 ID unici nel dataset EmoSign.

Caricamento del dataset locale con sentiment...
Dataset locale caricato con successo.
Trovati 2118 ID unici nel dataset locale.


In [17]:
# Step 3: Trovare le corrispondenze ("Golden Labels")
if "df_local" in locals() and not df_local.empty and emosign_ids:
    # Trova gli ID in comune tra i due dataset
    golden_ids = local_ids.intersection(emosign_ids)
    print(f"\nTrovate {len(golden_ids)} corrispondenze ('golden labels').")

    # Filtra il DataFrame locale per mantenere solo le righe con gli ID corrispondenti
    golden_labels_df = df_local[df_local["utterance_id"].isin(golden_ids)].copy()

    # Rimuovi la colonna 'utterance_id' di supporto se non più necessaria
    golden_labels_df.drop(columns=["utterance_id"], inplace=True)

    print("\nPrime 5 righe delle 'golden labels':")
    print(golden_labels_df.head())
else:
    print(
        "\nImpossibile trovare le corrispondenze a causa di errori nel caricamento dei dati."
    )
    golden_labels_df = pd.DataFrame()


Trovate 200 corrispondenze ('golden labels').

Prime 5 righe delle 'golden labels':
      video_name                                            caption   emotion
26  83664512.mp4  I was like, “Oh, wow, that`s fine.” He wanted ...  Positive
31     25328.mp4  There are so many things that are wrong with t...  Negative
38    253715.mp4  Then they would ask the HIT group about the ca...  Negative
39    255438.mp4  Or they will ask another group SMASHED about t...  Negative
40    253816.mp4  Or ask the COLLIDED group about the car and if...  Negative


In [18]:
# Step 4: Salvare il nuovo DataFrame
if not golden_labels_df.empty:
    # output_path = "../data/processed/golden_label_sentiment.csv" # Vecchio path
    output_path = "../data/processed/golden_label_sentiment_with_neutral.csv"  # Nuovo path con neutrali
    golden_labels_df.to_csv(output_path, index=False)
    print(
        f"\nFile '{os.path.basename(output_path)}' salvato con successo in '{os.path.dirname(output_path)}'."
    )
    print(f"Il file contiene {len(golden_labels_df)} righe.")
else:
    print("\nNessun dato da salvare.")


File 'golden_label_sentiment_with_neutral.csv' salvato con successo in '../data/processed'.
Il file contiene 200 righe.


In [19]:
# Step 5: Salvare il dataset locale escludendo le "Golden Labels"
if "df_local" in locals() and not df_local.empty and "golden_ids" in locals():
    # Filtra il DataFrame locale per escludere le righe con gli ID "golden"
    df_without_golden = df_local[~df_local["utterance_id"].isin(golden_ids)].copy()

    # Rimuovi la colonna 'utterance_id' di supporto
    df_without_golden.drop(columns=["utterance_id"], inplace=True)

    # Definisci il percorso del nuovo file CSV
    # Prende il nome del file originale e aggiunge "_without_golden"
    # original_path = "../data/processed/asllrp_video_sentiment_data_0.34.csv" # Vecchio path
    original_path = "../data/processed/asllrp_video_sentiment_data_with_neutral_0.5.csv"  # Nuovo path con neutrali
    base, ext = os.path.splitext(original_path)
    output_path_without_golden = f"{base}_without_golden{ext}"

    # Salva il nuovo DataFrame
    df_without_golden.to_csv(output_path_without_golden, index=False)

    print(
        f"\nFile '{os.path.basename(output_path_without_golden)}' salvato con successo."
    )
    print(f"Contiene {len(df_without_golden)} righe (righe originali - golden labels).")
    print(
        f"Righe originali: {len(df_local)}, Golden Labels: {len(golden_ids)}, Righe rimanenti: {len(df_without_golden)}"
    )

else:
    print(
        "\nImpossibile creare il file senza golden labels a causa di errori precedenti o dati mancanti."
    )


File 'asllrp_video_sentiment_data_with_neutral_0.5_without_golden.csv' salvato con successo.
Contiene 1918 righe (righe originali - golden labels).
Righe originali: 2118, Golden Labels: 200, Righe rimanenti: 1918
